In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.cluster import KMeans
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
import mne

In [2]:
import numpy as np
import mne
import entropy as ent
import neurokit2 as nk
from scipy.signal import welch
from nolds import lyap_r
import antropy as ant

In [3]:
participantsInfo = pd.read_table('./ds004504/participants.tsv')

In [4]:
#Getting all the groups
A_sub = participantsInfo[participantsInfo["Group"] == "A"]["participant_id"].tolist()
C_sub = participantsInfo[participantsInfo["Group"] == "C"]["participant_id"].tolist()
F_sub = participantsInfo[participantsInfo["Group"] == "F"]["participant_id"].tolist()

In [20]:
def derivativesPath(sub): #dericatives is the preprocessed data
    return f"ds004504/{sub}/eeg/{sub}_task-eyesclosed_eeg.set"

freq_bands = {
    "Delta": (0.5, 4),
    "Theta": (4, 8),
    "Alpha": (8, 12),
    "Beta": (12, 30),
    "Gamma": (30, 40),
}

groups = {
    'Alzeimers': ,
    'Control': [],
    'Dementia': []
}

window_length = 1.5
step_size = .75 #50% sliding window

# *** this is the part where we would do a for loop per group, per subject in that group ***

In [34]:
#*this data would be from forloop variables
group = 'Alzeimers'
subject=0


In [36]:
raw_sub = mne.io.read_raw_eeglab(derivativesPath(A_sub[subject]), preload=True)
sfreq = raw_sub.info['sfreq']

In [37]:


start_times = np.arange(0, raw.times[-1] - window_length, step_size)
events = np.array([
    [int(t * sfreq), 0, 1] for t in start_times  # MNE event format
])

# Create epochs with a fixed window size
epochs = mne.Epochs(
    raw_sub, events, event_id=1,
    tmin=0, tmax=window_length,
    baseline=None, detrend=1, preload=True
)


Not setting metadata
798 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 798 events and 751 original time points ...
0 bad epochs dropped


In [ ]:
for bandname, (low, high) in freq_bands:
    for epoch_data in epochs.get_data(fmin=low, fmax=high):
        channels_avg = an_epoch.mean(axis=-1) #all the channels average at that frequency band
        for channelGroup in groups['Alzeimers']
        groups['Alzeimers'].append(epoch_A_0)

# channels_avg = epochs_A_0.get_data(fmin=0.1, fmax=20).mean(axis=-1)